# Trending Topic on Twitter live data using Spark Streaming

Import sparkContext & StreamingContext from PySpark library.

In [ ]:
import time
import json
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

Create a sparkContext<br>
Create Spark Streaming Context using SC (spark context). parameter 10 is the batch interval. <br>
Every 10 second the analysis will be done.

In [ ]:
# SparkContext(“local[1]”) would not work with Streaming bc 2 threads are required
sc = SparkContext("local[2]", "Twitter Demo")
ssc = StreamingContext(sc, 10) #10 is the batch interval in seconds
ssc.checkpoint("checkpoint_TwitterApp")

Connect to socket broker using ssc (spark streaming context)<br>
Host  (localhost) & port : 6782

In [ ]:
IP = "127.0.0.1"
Port = 6782
Dstream = ssc.socketTextStream(IP, Port)

aggregate_tags_count() is used to the overall sum of each intervals

In [ ]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

### Process the Stream:
1. Receives tweet message, stored in Dstream. **Input DStream**
2. splits the messages into words. **Apply transformation on DStream : flatMap**
3. filters all the words which start with a hashtag(#). **transformation : filter**
4. converts the words to lowercase. **transformation : map**
5. maps each tag to (word, 1). **transformation : map**
6. then reduces and counts occurrences of each hash tag. (updateStateByKey) hashtags = **output DStream**

In [ ]:
# split each tweet into words
hashtags = Dstream.flatMap( lambda text: text.split( " " ) ).filter( lambda word: word.lower().startswith("#") ).map( lambda word: ( word.lower()
, 1 ) ).updateStateByKey(aggregate_tags_count)

Sort the trends based on the counts in decreasing order

In [ ]:
author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))

Print the final analysis: top trends on streaming twitter data

In [ ]:
author_counts_sorted_dstream.pprint(10)

### Starting the Spark Streaming:
Spark Streaming code we have written till now will not execute, untill we start the ssc.<br>
ssc.start() will start the spark streaming context. This is the Action for the whole code. <br>
Now it'll create the lineage & DAG & do the lazy evaluation & start running the whole sequesnce of code.


In [ ]:
# You must start the Spark StreamingContext, and await process termination…
ssc.start()

awaitTermination() is very important to stop the SSC.<br> 
When we kill this python process then this signal will be sent to awaitTermination() function.<br> 
it will finally stop the spark streaming job.

In [ ]:
ssc.awaitTermination()